## Pytorch - MNIST 

### 1. Dataset loader 준비

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 128
learning_rate = 0.0002
epochs = 10


mnist_train = dset.MNIST("./data/", train=True, 
                         transform = transforms.ToTensor(),
                         target_transform=None, download=True)

mnist_test = dset.MNIST("./data/", train=False, 
                        transform=transforms.ToTensor(), 
                        target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size,
                                          shuffle = True, num_workers=2, drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size,
                                          shuffle = False, num_workers=2, drop_last = True)


## 2. Build Model

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 5), # in channels, out channels, kernel_size
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
        
    # 순전파 거치면서 결괏값 return
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1) # flatten
        out = self.fc_layer(out)
        return out
    

## 3. Train Model

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss() # softmax + Cross Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_arr = []
for i in range(num_epoch):
    # 불러온 train loader (batch size)만큼 학습
    for j, [image, label] in enumerate(train_loader): 
        x = image.to(device)
        y_ = label.to(device)

        # 기울기 초기화, 순전파 계산, loss 계산, 역전파 반영, 경사하강법 진행
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        if j%1000==0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3074, grad_fn=<NllLossBackward>)
tensor(0.2092, grad_fn=<NllLossBackward>)
tensor(0.1902, grad_fn=<NllLossBackward>)
tensor(0.0529, grad_fn=<NllLossBackward>)
tensor(0.1149, grad_fn=<NllLossBackward>)
tensor(0.0335, grad_fn=<NllLossBackward>)
tensor(0.0572, grad_fn=<NllLossBackward>)
tensor(0.0198, grad_fn=<NllLossBackward>)
tensor(0.0115, grad_fn=<NllLossBackward>)
tensor(0.0096, grad_fn=<NllLossBackward>)


## 4. Model Validation

In [16]:
correct = 0
total = 0

# 기울기 계산 안함 (no_grad())
with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device) # 정답 label
        
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        
        # total - 전체 test data 갯수, correct - 맞은것갯수
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print('Accuracy of Test Data: {}'.format(100*correct/total))
    

Accuracy of Test Data: 99.12860870361328
